# Census API Scratch

In [ ]:
state_list = ['18', '17', '26', '39', '55', '10', '11', '12', '13', '24', '37', '45', '51', '54']
df = pd.read_table('national_county.txt', sep = ',', converters={'01': lambda x: str(x), '001': lambda x: str(x)})
df = df[df['01'].isin(state_list)]

In [ ]:
df['county'] = df['01'] + df['001']

In [ ]:
acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B15002_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B25010_001E&for=block%20group:*&in=state:10%20county:001&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')
data = acs_results_2016.text
total_df = pd.read_json(data)
errors = []


# Query ACS Data from Census API
for i in range(1, len(df)):
    acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B15002_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B25010_001E&for=block%20group:*&in=state:{}%20county:{}&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4'.format(df.iloc[i,1], df.iloc[i,2]))
    print(df.iloc[i,1], df.iloc[i,2])
    if acs_results_2016.status_code != 204:
        data = acs_results_2016.text
        acs_df = pd.read_json(data)
        total_df = pd.concat([total_df, acs_df])
    else:
        errors.append([df.iloc[i,1], df.iloc[i,2]])


In [ ]:
# Clean ACS data frame and rename columns# Clean  
variables = {'B15002_001E': "total_pop_over25", 'B15003_022E': "bachelors", 'B15003_023E': "masters", 'B15003_024E': "professional", 'B15003_024E': "doctorate"
            'B25010_001E': "avg_hh_size"}
acs_df.columns = acs_df.iloc[0]
acs_df.drop([0], inplace = True)
acs_df.rename(columns = variables, inplace=True)